In [294]:
import pandas as pd
import numpy as np
import operator

Importo la base de datos de los pacientes, recogida en SPSS y pasada posteriormente a documento csv. Para su analisis

In [295]:
datos=pd.read_csv("Analisis_ano.csv",sep=";")

In [296]:
datos.head()

,Médico,Tipo_cápsula,Fecha_procedimiento,ID,Sexo,Nacimiento,Edad,Cintura,IMC,Tiempo_estudio,...,Patency,No_aporta_previo,Eco_abdominal,Rx_abdomen,Entero_TAC,Entero_RM,Tránsito_intes,Endoscopia_previa,Intro_cápsula,Complicaciones
0,1,1,5/14/2012,1,1,6/24/1925,86,,,,...,0,1,0,0,0,0,0,0,0,0
1,1,1,9/5/2012,2,1,1/12/1953,59,,,,...,0,1,0,0,0,0,0,0,0,0
2,1,1,10/17/2012,3,1,2/21/1951,61,,,,...,0,1,0,0,0,0,0,0,0,0
3,1,1,12/14/2013,4,1,10/1/1950,63,,,,...,0,1,0,0,0,0,0,0,0,0
4,1,1,10/25/2013,5,0,7/4/1935,78,,,,...,0,1,0,0,0,0,0,2,0,0


In [297]:
datos.shape

(251, 150)

In [298]:
datos.dtypes[:]

Médico                  int64
Tipo_cápsula            int64
Fecha_procedimiento    object
ID                      int64
Sexo                    int64
                        ...  
Entero_RM               int64
Tránsito_intes          int64
Endoscopia_previa      object
Intro_cápsula           int64
Complicaciones         object
Length: 150, dtype: object

In [299]:
columnas=datos.columns

In [300]:
incompletas=[]
for e in columnas:
    if datos[e].isnull().sum()>=1:
        incompletas.append(e)
print(incompletas)

[]


Como se ve no hay ninguna variable, con valores nulos en toda la base de datos

In [301]:
columnas=datos.columns
columnas.tolist()

['Médico',
 'Tipo_cápsula',
 'Fecha_procedimiento',
 'ID',
 'Sexo',
 'Nacimiento',
 'Edad',
 'Cintura',
 'IMC',
 'Tiempo_estudio',
 'T_estómago',
 'T_ID_total',
 'T_colon_total',
 'T_expulsión',
 'Expulsa_cápsula',
 'HDOO_oculta',
 'HDOO_mani',
 'Crohn',
 'Alt_tránsito',
 'Anemia',
 'Dolor_abd',
 'Estreñimiento',
 'Diarrea',
 'Vómitos',
 'Pérdida_peso',
 'EII',
 'Colono_imcp',
 'Cribado',
 'Revisión',
 'Otros_indica',
 'Hernia_hiato_esof',
 'RGE_esof',
 'Trast_tránsito_esof',
 'Erosiones_esof',
 'Barret_esof',
 'Esof_eosinofilica_esof',
 'Estenosis_extr_esof',
 'No_evaluable_esof',
 'No_informado_esof',
 'Normal_esof',
 'Normal_estom',
 'No_informado_estom',
 'No_evaluable_estom',
 'Hernia_hiato_estom',
 'Trast_motilidad_estom',
 'Erosiones_estom',
 'Gastritis_estom',
 'Pólipo_estom',
 'Otros_estom',
 'No_informado_del',
 'No_evaluable_del',
 'Normal_del',
 'Hz_yeyuno',
 'Yeyunitis',
 'Erosiones_yey',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Hz_íleon',
 'Ileitis

In [302]:
#Borro la variable Dianoscitos porque sería un hallazgo compatible con la normalidad.
datos.drop(columns=['Dianocitos'],axis=0, inplace=True)

Uno en una única variable las hernias de hiato de esófago y estomago. Al ser el mismo diagnostico, simplemente puesto
en esófago o estomago según el momento de la cápsula en el que se diagnosticaba. Quedando unificada en la variable "Hernia_hiato_total"

In [303]:
def hernia(fila):
    a='Hernia_hiato_esof'
    b='Hernia_hiato_estom'
    if fila[a]==fila[b]:
        return fila[a]
    elif fila[a]<fila[b]:
        return fila[b]
    else:
        return fila[a]

In [304]:
datos["Hernia_hiato_total"]=datos.apply(lambda fila: hernia(fila),axis=1)

Compruebo que la reoorganización se ha realizado correctamente

In [305]:
comp=datos[['Hernia_hiato_esof','Hernia_hiato_estom',"Hernia_hiato_total"]].sort_values(by="Hernia_hiato_total"),
ascending=True
comp

(     Hernia_hiato_esof  Hernia_hiato_estom  Hernia_hiato_total
 0                    0                   0                   0
 150                  0                   0                   0
 151                  0                   0                   0
 152                  0                   0                   0
 153                  0                   0                   0
 ..                 ...                 ...                 ...
 84                   0                   1                   1
 212                  0                   1                   1
 210                  1                   0                   1
 188                  0                   1                   1
 125                  1                   0                   1
 
 [251 rows x 3 columns],)

Tras asegurarme, elimino las dos variables sobrantes

In [306]:
datos.drop(columns=['Hernia_hiato_esof','Hernia_hiato_estom'],axis=0, inplace=True)

# 1) Porcentaje de pacientes que vinieron con síntomas de dolor abdominal y tuvieron hallazgos en la cápsula. Diferenciando entre el centro en el que fue realizada la prueba.

Para ello, agrupo primero todas las varibles que representan hallazgos en la cápsula endoscopica. Los agrupo en dos 
tipos:
- Hallazgos que permiten tratamiento sin la necesidad de pruebas complementarias
- Hallazgos que harían necesaria la realización de pruebas complementaris para el diagnostico completo.

In [307]:
hz_Tta=["Hernia_hiato_total",
 'RGE_esof',
 'Erosiones_esof',
 'Erosiones_estom',
 'Pólipo_estom',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Diverticulo_íleon',
'Erosiones_del',
 'Diverticulo_del',
 'Enteritis_del',
 'Pólipo_del',
 'Otros_del',
 'Hemorroides',
 'Diverticulo_col',
 'Diverticulo_C_drch',
 'Diverticulo_sig',
 'Diverticulo_C_izq',
 'Sigmoiditis',
 'Proctitis',
 'Angiodisplasia_col',
 'Pólipo_col',
 'Pólipo_C_drch',
 'Pólipo_C_asce',
 'Pólipo_C_trans',
 'Pólipo_C_izq',
 'Pólipo_C_desc',
 'Pólipo_ciego',
 'Pólipo_sig',
 'Pólipo_recto',
 'Melanosis',
 'Otras_col']

In [308]:
hz_prueba=['Trast_tránsito_esof',  'Barret_esof',  'Esof_eosinofilica_esof',  'Estenosis_extr_esof',  
           'Trast_motilidad_estom','Gastritis_estom',  'Yeyunitis',  'Erosiones_yey',  'Ileitis', 
           'Erosiones_íleon',  'Pólipo_íleon',  'Erosiones_col', 'Otros_estom',]

En cuanto a las variables otros hallazgos de estómago, delgado y colon decido incluirlas en hz_Tta porque en su
mayoria son de este tipo, aunque según resultados posteriores decidire si sacarlas, en base a posibles alteraciones 
en el analisis final.

In [309]:
cidma_dolor = datos[(datos['Médico']==0) & 
                (datos['Dolor_abd']==1)]

In [328]:
cidma_dolor.shape #Número de pacientes con dolor abdominal del centro CIDMA

(53, 149)

In [311]:
cidma_dolor["Hz_Tta"] = cidma_dolor[hz_Tta].sum(axis=1)

53

In [332]:
len(cidma_dolor[cidma_dolor["Hz_Tta"]>=1])#Número de pacientes con hallazgos para futuras terapías

39

In [329]:
cidma_dolor["Hz_Prueba"] = cidma_dolor[hz_prueba].sum(axis=1) 

/usr/local/Cellar/ipython/7.7.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [330]:
len(cidma_dolor[cidma_dolor["Hz_Prueba"]>=1]) #Número de pacientes con hallazgos para futuros estudios

33

In [340]:
pct_normal=cidma_dolor[(cidma_dolor["Hz_Tta"]==0)&
(cidma_dolor["Hz_Prueba"]==0)]                     #Número de pacientes con hallazgos compatibles con la normalidad
len(pct_normal)

6

In [341]:
no_evaluable=['No_evaluable_esof','No_evaluable_estom','No_evaluable_del','No_evaluable_col']

In [365]:
no_eva=[]
for e in no_evaluable:
    no_eva=cidma_dolor[cidma_dolor[e].isin([1])]
len(no_eva)

11

In [370]:
for e in no_eva.index:
    for i in pct_normal.index:
        if e==i:
            print(e)

154


In [372]:
no_eva[no_evaluable].loc[154]

No_evaluable_esof     0
No_evaluable_estom    0
No_evaluable_del      0
No_evaluable_col      1
Name: 154, dtype: int64

Ahora vere cuales son los pricipales hallazgos de dichos pacientes

In [178]:
dic={}
for e in hz_Tta:
    b=cidma_dolor[e].value_counts()
    dic[e]=b
    

In [ ]:
cidma_dolor["Hernia_hiato_total"].value_counts()

In [205]:
dic["Hernia_hiato_total"]=pd.Series([29,14],index=[0,1])


In [225]:
hz_Tta_frec={}
for e in hz_Tta:
    if len(dic[e])>1:
        hz_Tta_frec[e]=dic[e][1]

In [230]:
sorted(hz_Tta_frec.items(), key=operator.itemgetter(1),reverse=True)

[('Diverticulo_col', 16),
 ('Hernia_hiato_total', 14),
 ('Hemorroides', 14),
 ('RGE_esof', 9),
 ('Otras_col', 8),
 ('Diverticulo_C_izq', 7),
 ('Diverticulo_sig', 6),
 ('Erosiones_estom', 5),
 ('Pólipo_estom', 5),
 ('Pólipo_col', 5),
 ('Otros_del', 4),
 ('Diverticulo_C_drch', 4),
 ('Linfagiectasias_yey', 3),
 ('Erosiones_del', 3),
 ('Enteritis_del', 3),
 ('Melanosis', 3),
 ('Angiodisplasia_col', 2),
 ('Erosiones_esof', 1),
 ('Diverticulo_íleon', 1),
 ('Pólipo_C_drch', 1),
 ('Pólipo_C_izq', 1),
 ('Pólipo_ciego', 1),
 ('Pólipo_sig', 1),
 ('Pólipo_recto', 1)]

In [110]:
uruguay_dolor = datos[(datos['Médico']==1) & 
                (datos['Dolor_abd']==1)]

In [111]:
len(uruguay_dolor["Médico"]) #Número de pacientes con dolor abdominal en el hospital Italiano

8

In [112]:
uruguay_dolor[uruguay_dolor[hz_Tta]==1].index

Int64Index([45, 65, 66, 99, 152, 164, 173, 250], dtype='int64')

In [113]:
uruguay_dolor[uruguay_dolor[hz_prueba]==1].index

Int64Index([45, 65, 66, 99, 152, 164, 173, 250], dtype='int64')

In [114]:
pregunta_1=pd.DataFrame({"Pacientes":[53,8],"Normal":[6,0],
                       "Hz Pruebas":[33,],"Hz Tratamiento":[39,],"No evaluable":[11,]},index=["Cidma","Uruguay"])
pregunta_1

,Pacientes,Normal,Hz Pruebas,Hz Tratamiento
Cidma,53,0,53,53
Uruguay,8,0,8,8
